In [3]:
from tqdm import tqdm
import os
import json
import random

In [4]:
apikey = ''

In [5]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {apikey}',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
        'model': 'gpt-4-1106-preview',
    }

    response = requests.post(
        'https://api.openai.com/v1/chat/completions',
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [6]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """
    return prompt.strip()

In [9]:
i = 0
shots = random.sample(arange - {i}, 1)
prompts = []
for no, s in enumerate(shots):
    prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts)
print(prompt)

Contoh soalan 1
objektif: Bahagian yang di dalam kurungan dalam ayat-ayat yang berikut mungkin mengandungi kesalahan bahasa dan mungkin juga tidak. Tandakan sama ada A, B atau C jika ayat itu mengandungi kesalahan dan tandakan D jika tiada kesalahan.
soalan: Halimah percaya (percaya benar) anaknya tidak terbabit dengan tuduhan jenayah kolar putih.
A. sungguh percaya benar
B. teramat percaya benar
C. sangat percaya benar
D. percaya benar
jawapan: D

objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.
A. hartawan
B. dermawan
C. bangsawan
D. sukarelawan


In [10]:
r = curl(prompt)

In [11]:
r.json()['choices'][0]['message']['content']

'jawapan: D'

In [12]:
for i in tqdm(range(len(questions))):
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|███████████████████████████████████████████| 54/54 [00:54<00:00,  1.01s/it]


In [16]:
import json

with open('output-1shot.json', 'w') as fopen:
    json.dump(questions, fopen)

In [17]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

54

In [18]:
correct = 0
for q in filtered:
    correct += q['output'].replace('jawapan:', '').split('.')[0].strip() == q['jawapan']
(correct / len(filtered)) * 100

66.66666666666666